In [6]:
import os
import pickle5 as pickle
import json
import utils
import numpy as np
import pandas as pd
import pyarrow as pa
import networkx as nx

from tqdm import tqdm

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialize Sherlock Model

In [7]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:02.651290 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:01.878260 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.000464 seconds.


[nltk_data] Downloading package punkt to /home/aristotle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aristotle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Initialize the pre-trained sherlock model with its default parameters 
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

# Use Sherlock to predict the semantic types for each cell node

## Helper Functions

In [10]:
def get_column_node_to_semantic_type_dict(data_dir):
    '''
    Given a directory `data_dir` containing a list of csv (i.e., tables) files use Sherlock
    to predict the semantic type for each column node (i.e, for each column in each specified table)
    '''
    # Dictionary mapping each column node to the predicted label using Sherlock
    column_node_to_semantic_type_dict = {}

    for filename in tqdm(os.listdir(data_dir)):
        df = pd.read_csv(data_dir+filename, keep_default_na=False, dtype=str)
        column_names=df.columns
        
        # Convert dataframe into a pandas series to be used as input for Sherlock
        rows = [df[column_name].tolist() for column_name in column_names]
        data_series = pd.Series(rows, name='values')
        
        # Extract features for the current table and save them at temporary.csv
        a = extract_features("../temporary.csv", data_series)
        feature_vectors = pd.read_csv("../temporary.csv", dtype=np.float32)
        
        # Use Sherlock to predict the semantic types for each column
        predicted_labels = model.predict(feature_vectors, "sherlock")
        for column_name, label in zip(column_names, predicted_labels):
            column_node_to_semantic_type_dict[column_name+'_'+filename] = label

        print(column_names, predicted_labels)

    return column_node_to_semantic_type_dict

def get_cell_node_to_semantic_type(graph, column_node_to_semantic_type_dict):
    '''
    Return a dictionary mapping each cell value in a graph to the list of its semantic types found using
    Sherlock.
    '''
    # Extract all cell nodes from the graph
    cell_nodes = {n for n, d in graph.nodes(data=True) if d['type']=='cell'}
    cell_node_to_semantic_type_dict = {}

    # For each cell node find all the column nodes it is connected to and extract their semantic types
    for node in cell_nodes:
        semantic_types = set()
        column_nodes = utils.graph_helpers.get_attribute_of_instance(graph, instance_node=node)
        for column_node in column_nodes:
            semantic_types.add(column_node_to_semantic_type_dict[column_node])
        
        cell_node_to_semantic_type_dict[node]=semantic_types
 
    return cell_node_to_semantic_type_dict

def get_predicted_homographs(cell_node_to_semantic_type_dict):
    '''
    Returns a set of the cell node that were predicted to be homographs 
    (i.e., they were assigned to more than 1 semantic type)
    '''
    predicted_homographs = set()
    for cell_node in cell_node_to_semantic_type_dict:
        if len(cell_node_to_semantic_type_dict[cell_node]) > 1:
            predicted_homographs.add(cell_node)
    return predicted_homographs

## Run Sherlock on the Synthetic Benchmark (SB)

In [11]:
data_dir="data/synthetic_benchmark/"
graph = pickle.load(open('graph_representations/synthetic_benchmark/bipartite.graph', 'rb'))

# Groundtruth Homographs
gt = pd.read_pickle('ground_truth/synthetic_example_groundtruth_dict.pickle')
gt_homographs = set([val for val in gt if gt[val]=='homograph'])

# Find the semantic types for each column
column_node_to_semantic_type_dict = get_column_node_to_semantic_type_dict(data_dir)

# Find the semantic types for each cell node
cell_node_to_semantic_type_dict = get_cell_node_to_semantic_type(graph, column_node_to_semantic_type_dict)
predicted_homographs = get_predicted_homographs(cell_node_to_semantic_type_dict)

  0%|          | 0/13 [00:00<?, ?it/s]

Exporting 1588 column features


  8%|▊         | 1/13 [00:00<00:07,  1.55it/s]

Index(['animal_name', 'animal_scientific_name', 'country'], dtype='object') ['species' 'species' 'country']
Exporting 1588 column features


 23%|██▎       | 3/13 [00:01<00:04,  2.35it/s]

...

Index(['full_name', 'credit_card_type', 'credit_card', 'email_address'], dtype='object') ['name' 'class' 'address' 'address']
Exporting 1588 column features



 85%|████████▍ | 11/13 [00:04<00:00,  2.76it/s]

Index(['city', 'country'], dtype='object') ['city' 'country']
Exporting 1588 column features


100%|██████████| 13/13 [00:05<00:00,  2.51it/s]

Index(['plant_name', 'plant_scientific_name', 'plant_family', 'country'], dtype='object') ['name' 'name' 'family' 'country']
Exporting 1588 column features
Index(['country', 'country_code'], dtype='object') ['country' 'country']


In [12]:
precision = len(gt_homographs & predicted_homographs) / len(predicted_homographs)
recall = len(gt_homographs & predicted_homographs) / len(gt_homographs)
f1_score = (2* precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)

Precision: 1.0
Recall: 0.6909090909090909
F1-Score: 0.8172043010752689


In [13]:
print("GT homographs not predicted by Sherlock:", gt_homographs - predicted_homographs)

GT homographs not predicted by Sherlock: {'Costanza', 'Crossfire', 'Christophe', 'Nadine', 'Leandra', 'Jimmy', 'Berkeley', 'Duff', 'Vinson', 'Reid', 'Smitty', 'Heather', 'Garvey', 'Conroy', 'Else', 'Charity', 'Elan'}


## Run Sherlock on TUS Benchmark

In [ ]:
data_dir="data/TUS/csvfiles/"
graph = pickle.load(open('graph_representations/TUS/bipartite.graph', 'rb'))

# Groundtruth Homographs
with open("ground_truth/groundtruth_TUS_short_format.pickle", "rb") as fh:
    gt = pickle.load(fh)
gt_homographs = set([val for val in gt if gt[val]=='homograph'])

# Find the semantic types for each column
column_node_to_semantic_type_dict = get_column_node_to_semantic_type_dict(data_dir)

# Find the semantic types for each cell node
cell_node_to_semantic_type_dict = get_cell_node_to_semantic_type(graph, column_node_to_semantic_type_dict)
predicted_homographs = get_predicted_homographs(cell_node_to_semantic_type_dict)